In [1]:
!git clone https://github.com/ggerganov/llama.cpp || echo 'Exists!'
!pip install -q torch torchvision huggingface-hub langchain

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Exists!

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
os.chdir('llama.cpp')
!make -s -i 2>&1
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir ./models --local-dir-use-symlinks False --quiet
!./main -m ./models/mistral-7b-instruct-v0.2.Q4_K_M.gguf -t 8 -n 128 -p 'Q: Who is the founder of python language?' 2>&1 | egrep "llama_print_timings|llm_load_tensors"

I llama.cpp build info: 
I UNAME_S:   Darwin
I UNAME_P:   arm
I UNAME_M:   arm64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_DARWIN_C_SOURCE -DNDEBUG -DGGML_USE_ACCELERATE -DACCELERATE_NEW_LAPACK -DACCELERATE_LAPACK_ILP64 -DGGML_USE_METAL  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -Wunreachable-code-break -Wunreachable-code-return -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_DARWIN_C_SOURCE -DNDEBUG -DGGML_USE_ACCELERATE -DACCELERATE_NEW_LAPACK -DACCELERATE_LAPACK_ILP64 -DGGML_USE_METAL  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread   -Wunreachable-code-break -Wunreachable-code-return -Wmissing-prototypes -Wextra-semi
I NVCCFLAGS:  
I LDFLAGS:   -framework Accelerate -framework Foundation -framework Metal -

In [3]:
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install -q llama-cpp-python


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_community.llms import LlamaCpp

n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512

llm = LlamaCpp(
    model_path="models/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,
    callback_manager=None
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

In [5]:
%time output = llm.invoke("Q: Who is the founder of python language? ", max_tokens=32)
print(output)

CPU times: user 10.4 s, sys: 1.29 s, total: 11.7 s
Wall time: 2.46 s

A: Python was created by Guido van Rossum and first released in 1994. Guido remains a key figure in the development and



llama_print_timings:        load time =     714.35 ms
llama_print_timings:      sample time =       2.68 ms /    32 runs   (    0.08 ms per token, 11931.39 tokens per second)
llama_print_timings: prompt eval time =     714.33 ms /    12 tokens (   59.53 ms per token,    16.80 tokens per second)
llama_print_timings:        eval time =    1679.34 ms /    31 runs   (   54.17 ms per token,    18.46 tokens per second)
llama_print_timings:       total time =    2435.17 ms /    43 tokens


In [6]:
n_gpu_layers = 33  # Metal set to 33(Max layer).
n_batch = 512

llm = LlamaCpp(
    model_path="models/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,
    callback_manager=None
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

In [7]:
%time output = llm.invoke("Q: Who is the founder of python language? ", max_tokens=32)
print(output)

ggml_metal_init: GPU name:   Apple M2 Pro
ggml_metal_init: GPU family: MTLGPUFamilyApple8  (1008)
ggml_metal_init: GPU family: MTLGPUFamilyCommon3 (3003)
ggml_metal_init: GPU family: MTLGPUFamilyMetal3  (5001)
ggml_metal_init: simdgroup reduction support   = true
ggml_metal_init: simdgroup matrix mul. support = true
ggml_metal_init: hasUnifiedMemory              = true
ggml_metal_init: recommendedMaxWorkingSetSize  = 11453.25 MB
ggml_backend_metal_buffer_type_alloc_buffer: allocated buffer, size =    64.00 MiB, ( 4367.83 / 10922.67)
llama_kv_cache_init:      Metal KV buffer size =    64.00 MiB
llama_new_context_with_model: KV self size  =   64.00 MiB, K (f16):   32.00 MiB, V (f16):   32.00 MiB
ggml_backend_metal_buffer_type_alloc_buffer: allocated buffer, size =     0.02 MiB, ( 4367.84 / 10922.67)
ggml_backend_metal_buffer_type_alloc_buffer: allocated buffer, size =    73.02 MiB, ( 4440.84 / 10922.67)
llama_new_context_with_model: graph splits (measure): 3
llama_new_context_with_model:

CPU times: user 67.6 ms, sys: 149 ms, total: 217 ms
Wall time: 1.39 s

A: Python was created by Guido van Rossum and first released in 1994 as a successor to the ABC programming language.



llama_print_timings:        load time =     350.83 ms
llama_print_timings:      sample time =       2.61 ms /    32 runs   (    0.08 ms per token, 12246.46 tokens per second)
llama_print_timings: prompt eval time =     350.80 ms /    12 tokens (   29.23 ms per token,    34.21 tokens per second)
llama_print_timings:        eval time =     996.91 ms /    31 runs   (   32.16 ms per token,    31.10 tokens per second)
llama_print_timings:       total time =    1387.88 ms /    43 tokens
